In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import math
import os
import os.path

In [2]:
def IG_scraper():
    input = menu()
    hashtag = input[0]
    length = input[1]
    name = input[2]
    selection = input[3]

    try:
        #Scraping links from the hashtag
        driver = webdriver.Chrome()
        driver.get('https://www.instagram.com/explore/tags/'+hashtag)
        driver.maximize_window()
        height = driver.execute_script("return document.body.scrollHeight")
        links=[]
        
        for i in range(length):
            try :
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                source = driver.page_source
                data=BeautifulSoup(source, 'lxml')
                body = data.find('body')
                span = body.find('span')
                for link in span.findAll('a'):
                    if re.match("/p", link.get('href')):
                        links.append('https://www.instagram.com'+link.get('href'))
                update_height = driver.execute_script("return document.body.scrollHeight")
                #Sometimes,the webpage failed to load the new content, then I scroll the page up to refresh it  
                if update_height == height:
                    driver.execute_script("window.scrollBy(0,-1000)") 
                height = update_height
                #Set each scrolling time for waiting the website to load the information
                time.sleep(1)
            except :
                pass 
            
        links2 = list(dict.fromkeys(links))
    
        if len(links2) != 0 :
            # Collect the chech-ins location and date from each links
            check_ins = []
            date = []
            driver2 = webdriver.Chrome()

            for url in links2 :
                try :
                    #Find the specific div of Check-ins
                    driver2.get(url)
                    page = driver2.page_source
                    soup = BeautifulSoup(page, 'lxml')
                    body = soup.find("body")
                    span = body.find("span")
                    section = span.find("section")
                    main = section.find("main")
                    div = main.find("div")
                    div2 = div.find("div")
                    article = div2.find("article")
                    header = article.find("header")
                    div3 = header.find("div",attrs={"class": "JF9hh"})
                    check_ins.append(div3.text)
        
                    #Find the specific div of Check-ins Date page = driver.page_source
                    div4 = article.find("div",attrs={"class": "eo2As"})
                    div5 = div4.find("div",attrs={"class": "k_Q0X NnvRN"})
                    time = div.find("time")
                    date.append(time.get("datetime"))
                except :
                    pass   
        else :
            return ('Error! No data or you input something wrong')
        
        join = {'Location':check_ins,'Date':date}
        df = pd.DataFrame(join)
        
        month = []
        year = []
        season = []
        for i in range (len(df.index)):
            year.append((df.loc[i,'Date'][0:4]))
            month.append((df.loc[i,'Date'][5:7]))  
            if df.loc[i,'Date'][5:7] in ['03','04','05'] :
                season.append('Spring')
            elif df.loc[i,'Date'][5:7] in ['06','07','08'] :
                season.append('Summer')
            elif df.loc[i,'Date'][5:7] in ['09','10','11'] :
                season.append('Fall')
            elif df.loc[i,'Date'][5:7] in ['12','01','02'] :
                season.append('Winter') 
        join = {'Month':month,'Year':year,'Season':season}
        df2 = pd.DataFrame(join)
        df = pd.concat([df,df2], axis=1)
        
    except :
        return ('Error! No data or you input something wrong1')
    
    if selection == 'y':
        #Clear all none location Check-ins
        df = df[df.Location != '']
        #Export file as csv
        df.to_csv(name +'.csv', index=False)
        print('Scraping Complete')
        
    elif selection == 'n':
        #Export file as csv
        df.to_csv(name +'.csv', index=False)
        print('Scraping Complete')
        

        
def menu():
    print('Welcome to IG Scrapter from hashtag')                                 
    print()
    print('You will need to specify the hashtag')
    hashtag = input('Enter the hashtag: ')
    print()
    print('You will need to specify number of scroll page (depend on how many post do the hashtag have)')
    print('***')
    print('Note! Time to scrape data directly relate to number of scoroll')
    print('Note! If number of scroll is too few, the you would not receive all data')
    print('Suggestion : 1 scroll could be scrape around 10 posts (would be less when deal with a lot of posts)')
    print('***')
    print('')
    print("Please don't close the browser while running")
    length = math.ceil(float(input('Enter the number of scroll: ')))
    print()
    print('You will need to specify the name of csv imported file')
    name = input('Enter the file name: ')
    print('') 
    print('Do you want to remove non checked-in data')
    selection = input('Make a selection (Y/N): ').lower().replace(" ","")    
    return hashtag, length, name, selection

IG_scraper()

Welcome to IG Scrapter from hashtag

You will need to specify the hashtag
Enter the hashtag: Halloween

You will need to specify number of scroll page (depend on how many post do the hashtag have)
***
Note! Time to scrape data directly relate to number of scoroll
Note! If number of scroll is too few, the you would not receive all data
Suggestion : 1 scroll could be scrape around 10 posts (would be less when deal with a lot of posts)
***

Please don't close the browser while running
Enter the number of scroll: 5

You will need to specify the name of csv imported file
Enter the file name: Halloween

Do you want to remove non checked-in data
Make a selection (Y/N): y
Scraping Complete
